# Distance Computation and KMeans

KMeans Tensorflow Exploration

Adapted from: https://gist.github.com/dave-andersen/265e68a5e879b5540ebc


Useful links:
    
    http://esciencegroup.com/2016/01/05/an-encounter-with-googles-tensorflow/
    http://learningtensorflow.com/lesson6/

In [1]:
import tensorflow as tf
import time
import tensorflow.contrib
sess = tf.InteractiveSession()

In [5]:
N=5
K=4

start = time.time()

points = tf.Variable(tf.random_uniform([N,2]))
cluster_assignments = tf.Variable(tf.zeros([N], dtype=tf.int64))

init_op = tf.initialize_all_variables()
init_op.run()


# Printing out Tensors

In [6]:
tf.Print(points, [points], "These are the points").eval()

array([[ 0.87300801,  0.77612352],
       [ 0.65893841,  0.41622484],
       [ 0.06771493,  0.03780019],
       [ 0.39849794,  0.18587363],
       [ 0.12047112,  0.13196683]], dtype=float32)

# Reshaping Points

In [8]:
rep_points = tf.reshape(tf.tile(points, [1, N]), [N, N, 2])
rep_points.eval()

array([[[ 0.87300801,  0.77612352],
        [ 0.87300801,  0.77612352],
        [ 0.87300801,  0.77612352],
        [ 0.87300801,  0.77612352],
        [ 0.87300801,  0.77612352]],

       [[ 0.65893841,  0.41622484],
        [ 0.65893841,  0.41622484],
        [ 0.65893841,  0.41622484],
        [ 0.65893841,  0.41622484],
        [ 0.65893841,  0.41622484]],

       [[ 0.06771493,  0.03780019],
        [ 0.06771493,  0.03780019],
        [ 0.06771493,  0.03780019],
        [ 0.06771493,  0.03780019],
        [ 0.06771493,  0.03780019]],

       [[ 0.39849794,  0.18587363],
        [ 0.39849794,  0.18587363],
        [ 0.39849794,  0.18587363],
        [ 0.39849794,  0.18587363],
        [ 0.39849794,  0.18587363]],

       [[ 0.12047112,  0.13196683],
        [ 0.12047112,  0.13196683],
        [ 0.12047112,  0.13196683],
        [ 0.12047112,  0.13196683],
        [ 0.12047112,  0.13196683]]], dtype=float32)

In [9]:
sum_squares = tf.reduce_sum(tf.square(rep_points - rep_points), 
                            reduction_indices=2)

In [10]:
sum_squares.eval()

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]], dtype=float32)

## Validation Against Native Python Version

In [19]:
p_np=points.eval()

In [20]:
from sklearn.metrics.pairwise import euclidean_distances
euclidean_distances(p_np, p_np)

array([[ 0.        ,  0.41875163,  1.09252834,  0.75733399,  0.9905805 ],
       [ 0.41875163,  0.        ,  0.70196187,  0.34769371,  0.6088922 ],
       [ 1.09252834,  0.70196187,  0.        ,  0.36241293,  0.10793781],
       [ 0.75733399,  0.34769371,  0.36241293,  0.        ,  0.28320459],
       [ 0.9905805 ,  0.6088922 ,  0.10793781,  0.28320459,  0.        ]], dtype=float32)

In [21]:
from scipy.spatial.distance import pdist
pdist(p_np)

array([ 0.41875155,  1.09252839,  0.75733398,  0.99058045,  0.70196183,
        0.34769371,  0.60889214,  0.36241294,  0.10793781,  0.28320462])

# KMeans

In [ ]:
# Silly initialization:  Use the first K points as the starting
# centroids.  In the real world, do this better.
centroids = tf.Variable(tf.slice(points.initialized_value(), [0,0], [K,2]))

# Replicate to N copies of each centroid and K copies of each
# point, then subtract and compute the sum of squared distances.
rep_centroids = tf.reshape(tf.tile(centroids, [N, 1]), [N, K, 2])
rep_points = tf.reshape(tf.tile(points, [1, K]), [N, K, 2])
sum_squares = tf.reduce_sum(tf.square(rep_points - rep_centroids), 
                            reduction_indices=2)


In [ ]:
MAX_ITERS = 1000


# Use argmin to select the lowest-distance point
best_centroids = tf.argmin(sum_squares, 1)
did_assignments_change = tf.reduce_any(tf.not_equal(best_centroids, 
                                                    cluster_assignments))

def bucket_mean(data, bucket_ids, num_buckets):
    total = tf.unsorted_segment_sum(data, bucket_ids, num_buckets)
    count = tf.unsorted_segment_sum(tf.ones_like(data), bucket_ids, num_buckets)
    return total / count

means = bucket_mean(points, best_centroids, K)

# Do not write to the assigned clusters variable until after
# computing whether the assignments have changed - hence with_dependencies
with tf.control_dependencies([did_assignments_change]):
    do_updates = tf.group(
        centroids.assign(means),
        cluster_assignments.assign(best_centroids))

init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

changed = True
iters = 0

while changed and iters < MAX_ITERS:
    iters += 1
    [changed, _] = sess.run([did_assignments_change, do_updates])

[centers, assignments] = sess.run([centroids, cluster_assignments])
end = time.time()
print ("Found in %.2f seconds" % (end-start)), iters, "iterations"
print "Centroids:"
print centers
print "Cluster assignments:", assignments